# Experiments on Graph-Generative-Models
In this notebook, we aim to evluate the performance of "GDSS" proposed in "Score-based Generative Modeling of Graphs via the System of Stochastic Differential Equations" (https://arxiv.org/pdf/2202.02514.pdf). The baseline model is tested on 3 datasets (Grid, Protein, 3D Point Cloud) and measured under 4 metrics (degree, clustering, orbit, graph spectrum).

### Experiment on GDSS
Here we immigrate the original terminal-executable GDSS codes into the notebook.

##### Change current directory.

In [1]:
import os
path = "./GDSS/"
os.chdir(path)

##### Install dependencies

In [2]:
!pip install -r requirements.txt
!conda install -c conda-forge rdkit=2020.09.1.0
!yes | pip install git+https://github.com/fabriziocosta/EDeN.git --user!

Defaulting to user installation because normal site-packages is not writeable
  Using cached molsets-0.3.1-py3-none-any.whl (51.6 MB)
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached pandas-1.1.5-cp36-cp36m-manylinux1_x86_64.whl (9.5 MB)
  Using cached easydict-1.9.tar.gz (6.4 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement kiwisolver==1.3.2 (from versions: 0.1.0, 0.1.1, 0.1.2, 0.1.3, 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.3.0, 1.3.1)
ERROR: No matching distribution found for kiwisolver==1.3.2
\ 
Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --user!
yes: standard output: Broken pipe


##### Assign dataset and seed

In [2]:
dataset = 'grid'
seed = 42

##### Generate dataset

In [3]:
!python data/data_generators.py --dataset $dataset

0 247 462
1 204 379
2 361 684
3 209 388
4 247 462
5 165 304
6 208 387
7 176 325
8 196 364
9 252 472
10 165 304
11 209 388
12 121 220
13 180 333
14 100 180
15 240 449
16 224 418
17 180 332
18 247 462
19 272 511
20 187 346
21 238 445
22 132 241
23 342 647
24 190 351
25 256 480
26 221 412
27 176 325
28 140 256
29 247 462
30 180 333
31 120 218
32 132 241
33 130 237
34 221 412
35 182 337
36 169 312
37 228 425
38 143 262
39 204 379
40 156 287
41 228 425
42 160 294
43 210 391
44 121 220
45 170 313
46 228 425
47 144 264
48 156 287
49 266 499
50 140 256
51 176 325
52 252 472
53 156 287
54 165 304
55 195 362
56 168 310
57 266 499
58 170 313
59 180 333
60 180 333
61 198 367
62 170 313
63 150 275
64 225 420
65 192 356
66 156 287
67 270 507
68 180 332
69 195 362
70 209 388
71 190 351
72 285 536
73 224 418
74 323 610
75 170 313
76 121 220
77 221 412
78 120 218
79 289 544
80 247 462
81 160 294
82 170 313
83 130 237
84 198 367
85 195 362
86 190 351
87 165 304
88 272 511
89 204 379
90 240 449
91 156 28

##### Train the GDSS model

In [10]:
import torch
import argparse
import time
from parsers.config import get_config
from trainer import Trainer
from sampler import Sampler, Sampler_mol


os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"
torch.cuda.empty_cache()

ts = time.strftime('%b%d-%H:%M:%S', time.gmtime())
config = get_config(dataset, seed)
trainer = Trainer(config) 
ckpt = trainer.train(ts)
if 'sample' in config.keys():
    config.ckpt = ckpt
    sampler = Sampler(config) 
    sampler.sample()

----------------------------------------------------------------------------------------------------
Make Directory grid/test in Logs
Sep07-17:13:27
----------------------------------------------------------------------------------------------------
[grid]   init=deg (5)   seed=42   batch_size=16
----------------------------------------------------------------------------------------------------
lr=0.01 schedule=True ema=0.999 epochs=2500 reduce=False eps=1e-05
(ScoreNetworkX)+(ScoreNetworkA=GCN,4)   : depth=5 adim=32 nhid=32 layers=7 linears=2 c=(2 8 4)
(x:VP)=(0.10, 1.00) N=1000 (adj:VP)=(0.20, 0.80) N=1000
----------------------------------------------------------------------------------------------------


[Epoch]:   0%|          | 0/2500 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/nethome/hsun409/anaconda3/envs/GDSS_env/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/nethome/hsun409/anaconda3/envs/GDSS_env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/mnt/284ac980-b350-4035-8e02-707f671ad89e/hsun409/Experiments-on-Graph-Generative-Models/GDSS/models/ScoreNetwork_A.py", line 138, in forward
    x, adjc = self.layers[_](x, adjc, flags)
  File "/nethome/hsun409/anaconda3/envs/GDSS_env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/mnt/284ac980-b350-4035-8e02-707f671ad89e/hsun409/Experiments-on-Graph-Generative-Models/GDSS/models/attention.py", line 103, in forward
    _x, mask = self.attn[_](x, adj[:,

##### Generate new graphs by the trained GDSS model

In [8]:
if dataset in ['QM9', 'ZINC250k']:
    sampler = Sampler_mol(config)
else:
    sampler = Sampler(config) 
sampler.sample()

./checkpoints/ego_small/Sep07-15:14:57.pth loaded
----------------------------------------------------------------------------------------------------
Make Directory ego_small/test in Logs
(Euler)+(None): eps=0.0001 denoise=True ema=False 
----------------------------------------------------------------------------------------------------
GEN SEED: 12


 
Round 0 : 80.46s
degree    : 0.022727
cluster   : 0.014434
orbit     : 0.013961
spectral  : 0.031346
